In [1]:
%matplotlib inline
import os
from collections import Counter
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

region_name = "Chr07_6770001_6920000"
SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
RAW_PATH = "/data2/zhoujb/project/cowpea_project/rawData/"
RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
OUT_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/GWAS/PL/{}/".format(region_name)

In [2]:
# 表型数据
raw_phenos = pd.read_excel(os.path.join(RAW_PATH, "Phenotypes for GWAS-upload public database.xlsx"), index_col="Accessions")
raw_phenos = raw_phenos.drop(columns="ID")
raw_phenos = raw_phenos.replace({"-":np.nan})
raw_phenos = raw_phenos.dropna(axis=0, how="all")
raw_phenos = raw_phenos.rename(columns={'HZ-Pod length':'HZ-PL', 'GZ-Pod length':'GZ-PL', 
                                        'HZ Pod Sugar content (mg/g)':'HZ-PSugar','GZ Pod Sugar content  (mg/g)':'GZ-PSugar', 
                                        'HZ Pod Starch content(mg/g)':'HZ-PStarch','GZ Pod Strach Content (mg/g)':'GZ-PStarch', 
                                        'HZ Pod Protein content(g/Kg)':'HZ-PProtein','GZ Pod protein  (g/Kg)':'GZ-PProtein', 
                                        'HZ Seed Sugar (mg/g)':'HZ-SSuger','GZ Seed Sugar (mg/g)':'GZ-SSuger', 
                                        'HZ Seed Starch (mg/g)':'HZ-SStarch','GZ Seed Starch (mg/g)':'GZ-SStarch', 
                                        'HZ Seed Protein (g/kg)':'HZ-SProtein','GZ Seed Protein (g/kg)':'GZ-SProtein', 
                                        'Pod shattering':'PS'})
target_df = raw_phenos[["HZ-PL"]].dropna()

acc_list = target_df.index.to_list()

#基因型数据
raw_snp = pd.read_table(os.path.join(SNP_PATH, "{}.txt".format(region_name)))
#raw_snp = raw_snp.fillna("N")
#raw_snp = raw_snp.replace({'Y':"N", 'R':"N", 'S':"N", 'W':"N", 'M':"N", 'K':"N"})

acc_inte = list(set(acc_list).intersection(raw_snp.columns))

target_df = target_df.loc[acc_inte]
raw_snp = raw_snp[['Chr', 'Pos']+acc_inte]

# Mapping IUPAC codes to possible alleles
iupac_to_alleles = {
    'A': ['A'],
    'T': ['T'],
    'C': ['C'],
    'G': ['G'],
    'R': ['A', 'G'],
    'Y': ['C', 'T'],
    'M': ['A', 'C'],
    'K': ['G', 'T'],
    'S': ['G', 'C'],
    'W': ['A', 'T'],
    'H': ['A', 'T', 'C'],
    'B': [ 'T', 'C', 'G'],
    'V': ['A', 'C', 'G'],
    'D': ['A', 'T', 'G'],
    'N': ['A', 'T', 'C', 'G']
}

def convert_iupac_to_alleles(iupac_code):
    return iupac_to_alleles.get(iupac_code, ['N'])

def most_frequent_values(input_list):
    # Count occurrences of each element
    counts = Counter(input_list)
    
    # Find the maximum frequency
    max_count = max(counts.values())
    
    # Find all elements with the maximum frequency
    most_frequent = [item for item, count in counts.items() if count == max_count]
    
    return most_frequent[0]

# Create VCF header
vcf_header = [
    '##fileformat=VCFv4.2',
    '##source=custom_script',
    '##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
    #'#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t' + '\t'.join(["{}_{}".format(x, x) for x in acc_inte])
    '#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t' + '\t'.join(acc_inte)
]

# Create a list to store VCF data lines
vcf_lines = []
# Iterate through each SNP (row in the original table)
for index in raw_snp.index:
    row_values = raw_snp.loc[index].values
    chrom = str(int(row_values[0][-2:]))
    pos = str(row_values[1])
    snp_id = str(row_values[1])
    
    # Collect unique alleles
    alleles_list = []
    for val in row_values[2:]:
        if val == "N":
            continue
    
        alleles = convert_iupac_to_alleles(val)
        alleles_list.extend(alleles)

    most_freq_val = most_frequent_values(alleles_list)
    ref = most_freq_val

    alleles_list = list(set(alleles_list))
    alleles_list.remove(ref)
    alt = ','.join(alleles_list)
        
    # Generate the genotype data for each sample
    genotypes = []
    for val in row_values[2:]:
        if val == 'N':
            genotypes.append('./.')
        else:
            alleles = convert_iupac_to_alleles(val)
            if len(alleles) > 1:
                if ref in alleles:
                     genotypes.append('0/1')
                else:
                    genotypes.append('1/2')
            else:
                if alleles[0] == ref:
                    genotypes.append('0/0')
                elif alleles[0] == alt:
                    genotypes.append('1/1')
                else:
                    genotypes.append('0/1')

    # Create the VCF line
    qual = '.'
    filter = 'PASS'
    info = '.'
    format_field = 'GT'
    vcf_line = f"{chrom}\t{pos}\t{snp_id}\t{ref}\t{alt}\t{qual}\t{filter}\t{info}\t{format_field}\t" + '\t'.join(genotypes)
    vcf_lines.append(vcf_line)

# Write VCF file
output_vcf_file = os.path.join(OUT_PATH, "{}.vcf".format(region_name))
with open(output_vcf_file, 'w') as vcf_file:
    for header_line in vcf_header:
        vcf_file.write(header_line + '\n')
    for vcf_line in vcf_lines:
        vcf_file.write(vcf_line + '\n')

print(f"VCF file saved as {output_vcf_file}")

with open(os.path.join(OUT_PATH, "PL.txt"), "w") as phen_file:
    phen_file.write(f"FID IID PL\n")

    
    for acc in acc_inte:
        fid = 0
        iid = acc
        phenotype = str(target_df.loc[acc].values[0])
        phen_file.write(f"{fid} {iid} {phenotype}\n")

print("ALL DONE")

VCF file saved as /data2/zhoujb/project/cowpea_project/basedXPXLR/GWAS/PL/Chr07_6770001_6920000/Chr07_6770001_6920000.vcf
ALL DONE


In [ ]:
plink2 --vcf Chr07_6770001_6920000.vcf --pheno PL.txt --pheno-name PL  --glm allow-no-covars --adjust --out PL

In [4]:
# Get top 10 position
pl_adjust = pd.read_table(os.path.join(OUT_PATH, "PL.PL.glm.linear.adjusted"))
top_snp_list = sorted(pl_adjust.head(10)["ID"].to_list())

# Read_file
hap_count = pd.read_table(os.path.join(RES_PATH, "Haplotype_Counts_Info.tsv"), index_col="Haplotype_id")
hap_base = pd.read_table(os.path.join(RES_PATH, "Haplotype_Basepair_Info.tsv"), index_col="Haplotype_ID")
hap_score = pd.read_table(os.path.join(RES_PATH, "Haplotype_Score_Info.tsv"), index_col="Haplotype_ID")
hap_id_df = pd.read_table(os.path.join(RES_PATH, "Genotype_by_Hap_ID.tsv"), index_col="Region_index")

# 表型数据
raw_phenos = pd.read_excel(os.path.join(RAW_PATH, "Phenotypes for GWAS-upload public database.xlsx"), index_col="Accessions")
raw_phenos = raw_phenos.drop(columns="ID")
raw_phenos = raw_phenos.replace({"-":np.nan})
raw_phenos = raw_phenos.dropna(axis=0, how="all")
raw_phenos = raw_phenos.rename(columns={'HZ-Pod length':'HZ-PL', 'GZ-Pod length':'GZ-PL', 
                                        'HZ Pod Sugar content (mg/g)':'HZ-PSugar','GZ Pod Sugar content  (mg/g)':'GZ-PSugar', 
                                        'HZ Pod Starch content(mg/g)':'HZ-PStarch','GZ Pod Strach Content (mg/g)':'GZ-PStarch', 
                                        'HZ Pod Protein content(g/Kg)':'HZ-PProtein','GZ Pod protein  (g/Kg)':'GZ-PProtein', 
                                        'HZ Seed Sugar (mg/g)':'HZ-SSuger','GZ Seed Sugar (mg/g)':'GZ-SSuger', 
                                        'HZ Seed Starch (mg/g)':'HZ-SStarch','GZ Seed Starch (mg/g)':'GZ-SStarch', 
                                        'HZ Seed Protein (g/kg)':'HZ-SProtein','GZ Seed Protein (g/kg)':'GZ-SProtein', 
                                        'Pod shattering':'PS'})
target_df = raw_phenos["HZ-PL"].dropna()

In [5]:
chr, start, end = region_name.split("_")

hap_count_sel = hap_count[(hap_count["Chr"]==chr)&(hap_count["Start"]==int(start))&(hap_count["End"]==int(end))].copy()
hap_score_sel = hap_score.loc[hap_count_sel.index].copy()
hap_score_sel = hap_score_sel[["Hap_Score.by_Total_pop"]].copy()

hap_count_score = pd.concat([hap_count_sel, hap_score_sel], axis=1)
hap_count_score = hap_count_score.sort_values(by=["Hap_Score.by_Total_pop", "Genotyped_prop"], ascending=[False, False])
hap_count_score["Total"] = hap_count_score['Genotyped_sample_num'].astype(str) + ' (' + (hap_count_score['Genotyped_prop'] * 100).round(2).astype(str) + '%)'
hap_count_score["VC"] = hap_count_score['VC_sample_num'].astype(str) + ' (' + (hap_count_score['VC_prop'] * 100).round(2).astype(str) + '%)'
hap_count_score["VL"] = hap_count_score['VL_sample_num'].astype(str) + ' (' + (hap_count_score['VL_prop'] * 100).round(2).astype(str) + '%)'
hap_count_score["G"] = hap_count_score['G_sample_num'].astype(str) + ' (' + (hap_count_score['G_prop'] * 100).round(2).astype(str) + '%)'
hap_count_score = hap_count_score[['Haplotype_class', 'Hap_Score.by_Total_pop', 'Total', 'VC', 'VL', 'G']].copy()
hap_count_score = hap_count_score.rename(columns={'Haplotype_class':"Class", 'Hap_Score.by_Total_pop':"Score"})

# Add pehn value
hap_id_sel = hap_id_df.loc[region_name]
for index in hap_count_score.index:
    hap_id_sample = hap_id_sel[hap_id_sel==index].index.to_list()
    phen_val_list = target_df.loc[list(set(target_df.index).intersection(hap_id_sample))].values
    if len(phen_val_list) > 1:
        mean_value = round(np.mean(phen_val_list), 2)
        sd_value = round(np.std(phen_val_list), 2)
        hap_count_score.loc[index, "PL"] = "{}  ± {}".format(mean_value, sd_value)
    elif len(phen_val_list) == 1:
        hap_count_score.loc[index, "PL"] = round(phen_val_list[0], 2)
    else:
        hap_count_score.loc[index, "PL"] = "NA"

# Add base
for index in hap_count_score.index:
    snp_pos = [int(x) for x in hap_base.loc[index, "SNP_Pos"].split(",")]
    hap_base_list = hap_base.loc[index, "Haplotype"].split(",")
    if len(snp_pos) != len(hap_base_list):
        raise ValueError("snp_pos not equal hap_base_list")

    for x, y in zip(snp_pos, hap_base_list):
        if x in top_snp_list:
            hap_count_score.loc[index, x] = y

hap_count_score = hap_count_score[['Class']+top_snp_list+['Total', 'VC', 'VL', 'G', 'Score', 'PL']]
hap_count_score.to_excel(os.path.join(OUT_PATH, "{}_PL.xlsx".format(region_name)))

In [15]:
chromosome, start_position, end_position = region_name.split("_")
start_position = int(start_position)
end_position = int(end_position)
snp_positions = top_snp_list

# Create the plot
plt.figure(figsize=(10, 0.1))

# Plot the genome region as a line
plt.plot([start_position, end_position], [1, 1], color='black')

# Plot the SNP positions as vertical bars
for snp in snp_positions:
    plt.plot([snp, snp], [0.99, 1.01], color='blue', marker='|', linewidth=0.1)

# Annotate the plot
#plt.title(f'Chromosome {chromosome}: Positions {start_position}-{end_position}')
#plt.xlabel('Position')

# Remove the y-axis
plt.yticks([])
plt.gca().axes.get_yaxis().set_visible(False)

plt.xticks([])
plt.gca().axes.get_xaxis().set_visible(False)

# Remove the top and right spines (border lines)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
#plt.gca().spines['bottom'].set_position(('outward', 10))



plt.savefig(os.path.join(OUT_PATH, "{}_PL.pdf".format(region_name)), format='pdf', bbox_inches='tight', transparent=True)